In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\User\miniconda3\envs\clip\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\User\miniconda3\envs\clip\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\User\miniconda3\envs\clip\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users

1. Load CLIP model

In [3]:
import torch
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pretrained OpenCLIP model
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-16',
    pretrained='openai'
)
model.to(device)
model.eval()

print("Loaded OpenCLIP model (open_clip library)")


c:\Users\User\miniconda3\envs\clip\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\miniconda3\envs\clip\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--timm--vit_base_patch16_clip_224.openai. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

Loaded OpenCLIP model (open_clip library)


# 2. Load CIFAR100

In [5]:
from torchvision.datasets import CIFAR100

root = os.path.expanduser("~/.cache")
train_ds = CIFAR100(root, download=True, train=True, transform=preprocess)
test_ds  = CIFAR100(root, download=True, train=False, transform=preprocess)

  3%|▎         | 4227072/169001437 [02:00<1:18:35, 34944.25it/s]


KeyboardInterrupt: 

# 3. Feature Extractor

In [ ]:
def extract_features(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    all_features, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Extracting features"):
            images = images.to(device)

            feats = model.encode_image(images)
            feats = feats / feats.norm(dim=-1, keepdim=True)   # normalize

            all_features.append(feats.cpu())
            all_labels.append(labels)

    X = torch.cat(all_features).numpy()
    y = torch.cat(all_labels).numpy()
    return X, y


# 4. Run feature extraction

In [ ]:
train_features, train_labels = extract_features(train_ds)
test_features, test_labels = extract_features(test_ds)

# 5. Train classifier

In [ ]:
clf = LogisticRegression(
    C=0.316,
    max_iter=1500,
    verbose=1,
    n_jobs=-1,
    random_state=0
)

clf.fit(train_features, train_labels)

preds = clf.predict(test_features)

# 6. Compute metrics (multi-class macro)

In [ ]:
accuracy  = accuracy_score(test_labels, preds)
precision = precision_score(test_labels, preds, average="macro")
recall    = recall_score(test_labels, preds, average="macro")  # sensitivity
f1        = f1_score(test_labels, preds, average="macro")

# ---- Specificity (macro) ----
cm = confusion_matrix(test_labels, preds)
specificities = []
for cls in range(cm.shape[0]):
    TN = np.sum(np.delete(np.delete(cm, cls, axis=0), cls, axis=1))
    FP = np.sum(np.delete(cm, cls, axis=0)[:, cls])
    specificity = TN / (TN + FP)
    specificities.append(specificity)

specificity_macro = np.mean(specificities)

# 7. Print results

In [ ]:
print("\n--- Evaluation Metrics ---")
print(f"Accuracy:      {accuracy:.4f}")
print(f"Precision:     {precision:.4f}")
print(f"Recall/Sens:   {recall:.4f}")
print(f"Specificity:   {specificity_macro:.4f}")
print(f"F1 Score:      {f1:.4f}")